# Explication du projet

La société Prêt à Dépenser est une société financière  qui propose des crédits à la consommation. Dans une logique de gestion du risque, l’entreprise souhaite mettre en place un outil de scoring de crédits capable d’estimer la probabilité qu’un client rembourse son emprunt. Cet outil permettra de classer automatiquement les demandes en deux catégories : les demandes de prêts peu risquées qui seront acceptées ou les demandes de prêts risquées qui seront refusées. Pour développer ce modèle de classification, la société a fourni un large pannel de données provenant de différentes sources et inclus des informations sociodemographiques et des données issues d'autres institutions financières.

Le projet visant à mettre en place cet outil de classification se décomposera en différentes étapes. Dans un premier temps, les différentes tables de données fournies par la société Prêt à dépenser seront explorées de manière à comprendre les données, a évaluer leur qualité et à identifier les variables pertinentes pour l'objectif du projet. Dans l'étape suivante, la table correspondant aux demandes de prêts sera enrichie via une jointure des variables identifiées précedemment et de nouvelles variables construites pour apporter des informations supplémentaires. La qualité de cette table enrichie sera évaluée dans la troisième étape. La quatrième étape permettra d'analyser la table enrichie pour identifier des premières tendances de relations entres les variables explicatives et entre les variables explicatives et la variable cible. Ensuite dans la cinquième étape, différents modèles de classification seront explorés et évalués. Dans la dernière étape, le modèle final retenu sera analysé plus en profondeur pour identifier les variables qui auront le plus implacté les résultats de prédiction du modèle

# Sommaire
- [1 Explication du projet](#Explication-du-projet)
- [2 Sommaire](#Sommaire)
- [3 Import](#Import-des-bibliothèques-et-des-données)
- [4 Exploration des tables de données](#Exploration-des-tables-de-données)
- [5 Enrichissement de la table principale](#Enrichissement-de-la-table-principale)
- [6 Analyse de la table](#Analyse-de-la-table)
- [7 Sauvegarde des tables finales](#Sauvegarde-des-tables-finales)
- [8 Conclusions](#Conclusions)


# Import des bibliothèques et des données

In [1]:
# Manipulation et analyse de données
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype

# Librairies utiles
import joblib
import sys
import os

# Sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

# Ajout du dossier parent dans les chemins d'import
sys.path.append(os.path.abspath(".."))

# Import d'une classe home made qui sera utilisée pour le prétraitement des variables liées au crédits actifs et approuvés
from Utils.Conditional_imputer import ConditionalImputer


In [2]:
# Chargement des données
App_train = pd.read_csv(f"../Output/Data_clients/App_train_final.csv", index_col='SK_ID_CURR')
Home_credit_desc = pd.read_csv(f"../Data/HomeCredit_columns_description.csv", index_col=0, encoding='Windows-1252')
pipe  = joblib.load(f"../Output/Pipelines/pipeline_to_deployed.joblib")


# Prétraitement donnée

In [3]:
# Ajout du dossier parent dans les chemins d'import
sys.path.append(os.path.abspath(".."))

# Import d'une classe home made qui sera utilisée pour le prétraitement des variables liées au crédits actifs et approuvés
from Utils.Conditional_imputer import ConditionalImputer

# Définition des variables à garder
features = [feat for feat in App_train.columns if feat != 'TARGET' and feat!='SK_ID_CURR']

# Séparation des données et de la cible
X_train = App_train[features]
Y_train = App_train['TARGET']

# Définition des variables liées aux crédits approuvés
num_cols_c_approuved = ['nb_credit_approuves', 'montant_mensuel_min', 
                       'montant_total', 'montant_mensuel_mean',
                       'n_jours_credit_mean', 'n_jours_credit_min', 'n_jours_credit_max',
                       'n_jours_retard_moyen', 'ratio_echeance_retard', 'ratio_approuve_demande']

# Définition des variables liées aux crédits actifs
num_cols_c_actifs = ['n_contrat_actif', 'montant_mensuel_actif', 'montant_dette_restante',
                     'n_jours_restant_mean', 'n_jours_restant_min', 'n_jours_restant_max']

# Définition des autres listes de variables
num_other_cols = [c for c in X_train.columns if is_numeric_dtype(X_train[c]) 
                  and c not in (num_cols_c_approuved + num_cols_c_actifs)]
cat_cols_label = [c for c in X_train.select_dtypes(include='object').columns if X_train[c].nunique() <= 2]
cat_cols_OneHot = [c for c in X_train.select_dtypes(include='object').columns if X_train[c].nunique() > 2]

# Défintion des pipelines associées aux variables numériques
num_var_approuve = Pipeline([
    ('conditional_imputer', ConditionalImputer(ref_col='nb_credit_approuves', variables=num_cols_c_approuved)),
    ('scaler', StandardScaler())
])
num_var_actif = Pipeline([
    ('conditional_imputer', ConditionalImputer(ref_col='n_contrat_actif', variables=num_cols_c_actifs)),
    ('scaler', StandardScaler())
])
num_other_var = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Définition du préprocesseur complet
preprocessor = ColumnTransformer([
    ('num_var_c_approuve', num_var_approuve, num_cols_c_approuved),
    ('num_var_c_actifs', num_var_actif, num_cols_c_actifs),
    ('num_other', num_other_var, num_other_cols),
    ('cat_cols_label', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_cols_label),
    ('cat_cols_OneHot', OneHotEncoder(handle_unknown='ignore'), cat_cols_OneHot)
])

# Affichage du nombre d'observations et de variables des données d'entrainement
X_train.shape

(246006, 86)

# Construction de la table de description des variables

## Table contenant les variables d'origine

In [4]:
# Extraction des noms de variables
Features = App_train.columns

# Exrtaction des noms de variables de la table application d'origine
tab_desc_en = Home_credit_desc[Home_credit_desc['Table']=='application_{train|test}.csv']

# Extraction des variables qui ont été conservées dans la table finale
tab_desc_en = tab_desc_en[tab_desc_en['Row'].isin(Features)]

# Conservation des variables Row et Descriptions
tab_desc_en = tab_desc_en[['Row', 'Description']]

# Ajout d'une variable pour indiquer la source de la variable
tab_desc_en['Source'] = 'Application'

# Visualisation de la table
tab_desc_en

,Row,Description,Source
2,TARGET,Target variable (1 - client with payment diffi...,Application
5,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application
6,CODE_GENDER,Gender of the client,Application
7,FLAG_OWN_CAR,Flag if the client owns a car,Application
8,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application
...,...,...,...
120,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,Application
121,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,Application
122,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,Application
123,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,Application


## Ajout des variables construites

In [5]:
# Liste des noms de variables ajoutées à la table
row = ['n_jours_retard_moyen', 'n_contrat_actif', 'montant_mensuel_min',
       'montant_mensuel_mean', 'montant_total', 'n_jours_credit_mean',
       'n_jours_credit_min', 'n_jours_credit_max', 'montant_dette_restante',
       'n_jours_restant_mean', 'n_jours_restant_min', 'n_jours_restant_max',
       'nb_credit_approuves', 'ratio_echeance_retard',
       'ratio_approuve_demande', 'montant_mensuel_actif',
       'Another type of loan', 'Cash loans', 'Consumer credit',
       'Consumer loans', 'Immo_credit', 'Microloan', 'Revolving loans',
       'business credit']

# Liste des descriptions associées aux variables ajoutées dans la table
description_eng = ['Average number of days of payment delay for previous loans',
              'Number of active lonas',
               'Minimum monthly installment amount of previous loans',
               'Average monthly installment amount of previous loans',
               'Total loan amount',
               'Average previous loan duration (in days)',
               'Minimum loan duration (in days)',
               'Maximum loan duration (in days)',
               'Remaining debt amount',
               'Average number of days remaining on loans',
               'Minimum number of days remaining on loans',
               'Maximum number of days remaining on loans',
               'Number of approved loans',
               'Ratio of overdue installments to total installments',
               'Ratio of approved loans to total applications',
               'Monthly installment amount for active loans',
               'Number of another loan category',
               'Number of cash loan products',
               'Number of consumer credit loans',
               'Number of consumer loan products',
               'Number of mortgage or real estate loans',
               'Number of microloan products',
               'Number of revolving credit products',
               'Number of business-related loans']

In [6]:
# Construction d'une table intermédiaire        
tab_desc_en_int = pd.DataFrame({'Row': row,
             'Description': description_eng})

# Ajout de la source des variables construites
tab_desc_en_int['Source']= 'Engineered'

# Concaténation de la table intermédiaire à la table principale
tab_desc_en = pd.concat([tab_desc_en, tab_desc_en_int])

# Correction des descriptions de deux variables
tab_desc_en.loc[tab_desc_en['Row']=='FLAG_WORK_PHONE', 'Description'] = "Did client provide work phone (1=YES, 0=NO)"
tab_desc_en.loc[tab_desc_en['Row']=='FLAG_EMP_PHONE', 'Description'] = "Did client provide employer phone (1=YES, 0=NO)"

# Visualisation de la table
tab_desc_en

,Row,Description,Source
2,TARGET,Target variable (1 - client with payment diffi...,Application
5,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application
6,CODE_GENDER,Gender of the client,Application
7,FLAG_OWN_CAR,Flag if the client owns a car,Application
8,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application
...,...,...,...
19,Consumer loans,Number of consumer loan products,Engineered
20,Immo_credit,Number of mortgage or real estate loans,Engineered
21,Microloan,Number of microloan products,Engineered
22,Revolving loans,Number of revolving credit products,Engineered


## Ajout du type de la variable

In [7]:
# Fonction pour mapper les type
def map_dtype(dtype):
    if "int" in str(dtype):
        return "int"
    elif "float" in str(dtype):
        return "float"
    elif dtype == "object" or "category" in str(dtype):
        return "cat"
    else:
        return "other"

In [8]:
# Création d'un dataframe contenant les types de chaque variable
var_types = App_train.dtypes.apply(map_dtype).reset_index()
var_types.columns = ['Row', 'Type']

# Jointure avec ta table des variables
tab_desc_en = tab_desc_en.merge(var_types, on='Row', how='left')

# Visualisation de la table
tab_desc_en

,Row,Description,Source,Type
0,TARGET,Target variable (1 - client with payment diffi...,Application,int
1,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application,cat
2,CODE_GENDER,Gender of the client,Application,cat
3,FLAG_OWN_CAR,Flag if the client owns a car,Application,cat
4,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application,cat
...,...,...,...,...
81,Consumer loans,Number of consumer loan products,Engineered,float
82,Immo_credit,Number of mortgage or real estate loans,Engineered,float
83,Microloan,Number of microloan products,Engineered,float
84,Revolving loans,Number of revolving credit products,Engineered,float


## Ajout des valeurs prises par les variables catégorielles

In [9]:
# Fonction pour extraire les valeurs uniques pour les variables catégorielles
def get_values(row):
    col = row['Row']
    if row['Type'] == 'cat':
        vals = App_train[col].dropna().unique()

        # tu peux limiter le nombre pour éviter des colonnes énormes
        return ", ".join(map(str, vals[:20])) + ("..." if len(vals) > 20 else "")
    else:
        return pd.NA

In [10]:
# Application
tab_desc_en['Values'] = tab_desc_en.apply(get_values, axis=1)

# Visualisation de la table
tab_desc_en

,Row,Description,Source,Type,Values
0,TARGET,Target variable (1 - client with payment diffi...,Application,int,<NA>
1,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application,cat,"Cash loans, Revolving loans"
2,CODE_GENDER,Gender of the client,Application,cat,"F, M"
3,FLAG_OWN_CAR,Flag if the client owns a car,Application,cat,"N, Y"
4,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application,cat,"N, Y"
...,...,...,...,...,...
81,Consumer loans,Number of consumer loan products,Engineered,float,<NA>
82,Immo_credit,Number of mortgage or real estate loans,Engineered,float,<NA>
83,Microloan,Number of microloan products,Engineered,float,<NA>
84,Revolving loans,Number of revolving credit products,Engineered,float,<NA>


## Info sur la selection ou non de la variable 

In [11]:
# Colonnes numériques et catégorielles avant preprocessing
num_cols_orig = num_cols_c_approuved + num_cols_c_actifs + num_other_cols
cat_cols_orig = {
    'label': cat_cols_label,
    'one_hot': cat_cols_OneHot
}

# Extraction des noms après preprocessing et feature_selection ---
feature_names_after_preprocessing = pipe.named_steps['preprocessing'].get_feature_names_out(X_train.columns)
mask = pipe.named_steps['feature_selection'].get_support()
selected_feature_names = feature_names_after_preprocessing[mask]
feature_names_clean = [f.split("__")[-1] for f in selected_feature_names]

# Mapping feature transformée -> feature originale et nom final ---
feature_mapping = {}
final_columns = []

for f_trans in feature_names_clean:
    # Identification de la feature originale
    if f_trans in num_cols_orig or f_trans in cat_cols_orig.get('label', []):
         f_orig = f_trans
    else:
        match_cat = next((f for f in cat_cols_orig.get('one_hot', []) if f_trans.startswith(f)), None)
        f_orig = match_cat if match_cat else f_trans

    # Nom final : pour les one-hot, on garde juste la catégorie
    if f_orig in cat_cols_orig.get('one_hot', []):
        f_final = f_trans.replace(f_orig + "_", "")
    else:
        f_final = f_trans
        
    feature_mapping[f_trans] = {'original': f_orig, 'final': f_final}
    final_columns.append(f_final)
    
# Récupération de tous les noms originaux dans le dictionnaire
original_features = [v['original'] for v in feature_mapping.values()]

# Liste des variables selectionnées
print(original_features)

['nb_credit_approuves', 'montant_mensuel_min', 'montant_total', 'montant_mensuel_mean', 'n_jours_credit_mean', 'n_jours_credit_min', 'n_jours_credit_max', 'n_jours_retard_moyen', 'ratio_echeance_retard', 'ratio_approuve_demande', 'n_contrat_actif', 'montant_mensuel_actif', 'montant_dette_restante', 'n_jours_restant_mean', 'n_jours_restant_max', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'REGION_POPULATION_RELATIVE', 'YEARS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE', 'Cash loans', 'Consumer credit', 'Consumer loans', 'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_EDUCATION_TYPE']


In [12]:
# Ajout d'une variable indiquant si la variable a été utilisée par le modèle
tab_desc_en['Utilisé'] = tab_desc_en['Row'].isin(original_features)

# Visualisation de la table
tab_desc_en

,Row,Description,Source,Type,Values,Utilisé
0,TARGET,Target variable (1 - client with payment diffi...,Application,int,<NA>,False
1,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application,cat,"Cash loans, Revolving loans",False
2,CODE_GENDER,Gender of the client,Application,cat,"F, M",True
3,FLAG_OWN_CAR,Flag if the client owns a car,Application,cat,"N, Y",True
4,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application,cat,"N, Y",False
...,...,...,...,...,...,...
81,Consumer loans,Number of consumer loan products,Engineered,float,<NA>,True
82,Immo_credit,Number of mortgage or real estate loans,Engineered,float,<NA>,False
83,Microloan,Number of microloan products,Engineered,float,<NA>,False
84,Revolving loans,Number of revolving credit products,Engineered,float,<NA>,False


## Ajout du type d'encodage

In [13]:
# Type
conditions = [
    tab_desc_en['Row'].isin(cat_cols_OneHot),
    tab_desc_en['Row'].isin(cat_cols_label),
    tab_desc_en['Row'].isin(num_cols_orig)
]

choices = ["OneHot", "Label", "Num"]

tab_desc_en['Type_Enc'] = np.select(conditions, choices, default="Unknown")

# Visualisation de la table
tab_desc_en

,Row,Description,Source,Type,Values,Utilisé,Type_Enc
0,TARGET,Target variable (1 - client with payment diffi...,Application,int,<NA>,False,Unknown
1,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,Application,cat,"Cash loans, Revolving loans",False,Label
2,CODE_GENDER,Gender of the client,Application,cat,"F, M",True,Label
3,FLAG_OWN_CAR,Flag if the client owns a car,Application,cat,"N, Y",True,Label
4,FLAG_OWN_REALTY,Flag if client owns a house or flat,Application,cat,"N, Y",False,Label
...,...,...,...,...,...,...,...
81,Consumer loans,Number of consumer loan products,Engineered,float,<NA>,True,Num
82,Immo_credit,Number of mortgage or real estate loans,Engineered,float,<NA>,False,Num
83,Microloan,Number of microloan products,Engineered,float,<NA>,False,Num
84,Revolving loans,Number of revolving credit products,Engineered,float,<NA>,False,Num


In [14]:
# Affichage des descriptions
for index, row in tab_desc_en.iterrows():
    print(f"- {row['Row']} : {row['Description']}")
    
tab_desc_en.to_csv(f"../Output/Variables/Variable_description_eng.csv", index=False)

- TARGET : Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)
- NAME_CONTRACT_TYPE : Identification if loan is cash or revolving
- CODE_GENDER : Gender of the client
- FLAG_OWN_CAR : Flag if the client owns a car
- FLAG_OWN_REALTY : Flag if client owns a house or flat
- CNT_CHILDREN : Number of children the client has
- AMT_INCOME_TOTAL : Income of the client
- AMT_CREDIT : Credit amount of the loan
- AMT_ANNUITY : Loan annuity
- NAME_TYPE_SUITE : Who was accompanying client when he was applying for the loan
- NAME_INCOME_TYPE : Clients income type (businessman, working, maternity leave,…)
- NAME_EDUCATION_TYPE : Level of highest education the client achieved
- NAME_FAMILY_STATUS : Family status of the client
- NAME_HOUSING_TYPE : What is the housing situation of the client (renting, living with parents, ...)
- REGION_POPULATION_RELATIVE : Normalized 

# Construction de la table de description en francais

In [15]:
description_fr = [
"Variable cible (1 = client avec des difficultés de paiement : il/elle a eu un retard de paiement de plus de X jours sur au moins une des Y premières mensualités du prêt dans notre échantillon, 0 = tous les autres cas)",
"Identification si le prêt est un crédit cash ou revolving",
"Sexe du client",
"Indicateur si le client possède une voiture",
"Indicateur si le client possède une maison ou un appartement",
"Nombre d’enfants du client",
"Revenu du client",
"Montant du crédit demandé",
"Annuité du prêt",
"Personne qui accompagnait le client lors de la demande de crédit",
"Type de revenu du client (salarié, indépendant, congé maternité, …)",
"Niveau d’éducation le plus élevé atteint par le client",
"Situation familiale du client",
"Situation de logement du client (locataire, vit chez ses parents, …)",
"Population normalisée de la région où vit le client (valeur élevée = région plus peuplée)",
"Nombre de jours entre la date de début de l’emploi actuel et la demande de prêt",
"Nombre de jours entre le changement d’adresse de résidence et la demande de prêt",
"Nombre de jours entre le changement de document d’identité et la demande de prêt",
"Le client a fourni un numéro de téléphone portable (1 = OUI, 0 = NON)",
"Le client a fourni un numéro de téléphone de son entreprise (1 = OUI, 0 = NON)",
"Le client a fourni un numéro de téléphone professionnel (1 = OUI, 0 = NON)",
"Le téléphone portable du client était joignable (1 = OUI, 0 = NON)",
"Le client a fourni un numéro de téléphone fixe (1 = OUI, 0 = NON)",
"Le client a fourni un email (1 = OUI, 0 = NON)",
"Notre notation de la région de résidence du client (1, 2, 3)",
"Jour de la semaine de la demande de crédit",
"Heure approximative de la demande de crédit",
"Indicateur si l’adresse permanente ≠ adresse de contact (1 = différente, 0 = identique, au niveau région)",
"Indicateur si l’adresse de contact ≠ adresse professionnelle (1 = différente, 0 = identique, au niveau région)",
"Indicateur si l’adresse permanente ≠ adresse de contact (1 = différente, 0 = identique, au niveau ville)",
"Indicateur si l’adresse de contact ≠ adresse professionnelle (1 = différente, 0 = identique, au niveau ville)",
"Score normalisé provenant d’une source de données externe",
"Score normalisé provenant d’une source de données externe",
"Nombre d’observations dans l’entourage social du client avec un défaut de paiement ≥ 30 jours",
"Nombre d’observations dans l’entourage social du client ayant effectivement eu un défaut ≥ 30 jours",
"Nombre de jours entre le changement de téléphone et la demande de prêt",
"Le client a fourni le document 2",
"Le client a fourni le document 3",
"Le client a fourni le document 4",
"Le client a fourni le document 5",
"Le client a fourni le document 6",
"Le client a fourni le document 7",
"Le client a fourni le document 8",
"Le client a fourni le document 9",
"Le client a fourni le document 10",
"Le client a fourni le document 11",
"Le client a fourni le document 12",
"Le client a fourni le document 13",
"Le client a fourni le document 14",
"Le client a fourni le document 15",
"Le client a fourni le document 16",
"Le client a fourni le document 17",
"Le client a fourni le document 18",
"Le client a fourni le document 19",
"Le client a fourni le document 20",
"Le client a fourni le document 21",
"Nombre de demandes d’information auprès du Bureau de crédit une heure avant la demande",
"Nombre de demandes d’information auprès du Bureau de crédit un jour avant la demande (hors dernière heure)",
"Nombre de demandes d’information auprès du Bureau de crédit une semaine avant la demande (hors dernier jour)",
"Nombre de demandes d’information auprès du Bureau de crédit un mois avant la demande (hors dernière semaine)",
"Nombre de demandes d’information auprès du Bureau de crédit trois mois avant la demande (hors dernier mois)",
"Nombre de demandes d’information auprès du Bureau de crédit un an avant la demande (hors trois derniers mois)",
"Nombre moyen de jours de retard de paiement sur les prêts précédents",
"Nombre de prêts actifs",
"Montant minimal des mensualités des prêts précédents",
"Montant moyen des mensualités des prêts précédents",
"Montant total des prêts précédents",
"Durée moyenne des prêts précédents (en jours)",
"Durée minimale des prêts précédents (en jours)",
"Durée maximale des prêts précédents (en jours)",
"Montant de la dette restante",
"Durée moyenne restante des prêts",
"Durée minimale restante des prêts",
"Durée maximale restante des prêts",
"Nombre de prêts approuvés",
"Ratio des mensualités en retard sur le total des mensualités",
"Ratio des prêts approuvés sur le total des demandes",
"Montant de la mensualité des prêts actifs",
"Nombre d’autres catégories de prêts",
"Nombre de prêts cash",
"Nombre de crédits à la consommation",
"Nombre de produits de crédit à la consommation",
"Nombre de prêts immobiliers ou hypothécaires",
"Nombre de micro-crédits",
"Nombre de crédits revolving",
"Nombre de prêts liés à une activité professionnelle"
]

In [16]:
# Copie de la table de description en anglais
tab_desc_fr = tab_desc_en.copy()

# Ajout des description en francais
tab_desc_fr['Description'] = description_fr

# Visualisation de la table
tab_desc_fr


,Row,Description,Source,Type,Values,Utilisé,Type_Enc
0,TARGET,Variable cible (1 = client avec des difficulté...,Application,int,<NA>,False,Unknown
1,NAME_CONTRACT_TYPE,Identification si le prêt est un crédit cash o...,Application,cat,"Cash loans, Revolving loans",False,Label
2,CODE_GENDER,Sexe du client,Application,cat,"F, M",True,Label
3,FLAG_OWN_CAR,Indicateur si le client possède une voiture,Application,cat,"N, Y",True,Label
4,FLAG_OWN_REALTY,Indicateur si le client possède une maison ou ...,Application,cat,"N, Y",False,Label
...,...,...,...,...,...,...,...
81,Consumer loans,Nombre de produits de crédit à la consommation,Engineered,float,<NA>,True,Num
82,Immo_credit,Nombre de prêts immobiliers ou hypothécaires,Engineered,float,<NA>,False,Num
83,Microloan,Nombre de micro-crédits,Engineered,float,<NA>,False,Num
84,Revolving loans,Nombre de crédits revolving,Engineered,float,<NA>,False,Num


In [17]:
tab_desc_fr.to_csv(f"../Output/Variables/Variable_description_fr.csv", index=False)

# Conclusions